In [7]:
import numpy as np
import pandas as pd

train = pd.read_csv('C:/Users/isang/OneDrive/Desktop/DL/Validation/train.csv')
test = pd.read_csv('C:/Users/isang/OneDrive/Desktop/DL/Validation/test.csv')
submission = pd.read_csv('C:/Users/isang/OneDrive/Desktop/DL/Validation/gender_submission.csv')


In [8]:
# Combine train and test datasets (excluding the 'Survived' column from train)
data_set = pd.concat((train.drop(['Survived'], axis = 1), test), axis = 0)

# Drop unnecessary or non-numeric columns
data_set = data_set.drop(['PassengerId', 'Name', 'Sex', 'Ticket', 'Cabin', 'Embarked'], axis = 1)
# Fill missing values with the mean of each column
data_set = data_set.fillna(data_set.mean())

# Split the combined dataset back into train and test sets
n_train = train.shape[0]
train_x, test_x = data_set[:n_train], data_set[n_train:]
train_y = train['Survived']

# Convert DataFrames to NumPy arrays
train_x = train_x[train_x.keys()].values
test_x = test_x[test_x.keys()].values
train_y = train_y.values

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset

class SimpleNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(5, 128)
        self.layer2 = nn.Linear(128, 256)
        self.layer3 = nn.Linear(256, 128)
        self.layer4 = nn.Linear(128, 1)

    def forward(self, x):
        # Apply first linear layer followed by ReLU activation
        x = self.layer1(x)
        x = F.relu(x)
        # Apply second linear layer followed by ReLU activation
        x = self.layer2(x)
        x = F.relu(x)
        # Apply third linear layer followed by ReLU activation
        x = self.layer3(x)
        x = F.relu(x)
        # Apply final linear layer
        x = self.layer4(x)
        # Apply sigmoid activation to output a value between 0 and 1
        x = torch.sigmoid(x)
        return x

class SimpleDataset(Dataset):
    def __init__(self, x, y):
        super().__init__()
        self.x = x
        self.y = y
        
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

In [10]:
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader, SubsetRandomSampler
from sklearn.model_selection import KFold

# Define number of folds for K-Fold Cross Validation
k_folds = 5
kfold = KFold(n_splits=k_folds, shuffle=True)

# Wrap training data in a PyTorch Dataset
simple_dataset = SimpleDataset(train_x, train_y)

# Lists to store validation loss and accuracy from each fold
loss_list = []
acc_list = []

# K-Fold Cross Validation loop
for fold, (train_ids, valid_ids) in enumerate(kfold.split(simple_dataset)):
    print(f'FOLD {fold}')
    print('--------------------------------')

    # Create samplers for training and validation sets
    train_subsampler = SubsetRandomSampler(train_ids)
    valid_subsampler = SubsetRandomSampler(valid_ids)
    
    # Create DataLoaders using the samplers
    train_loader = DataLoader(simple_dataset, batch_size=99, sampler=train_subsampler)
    valid_loader = DataLoader(simple_dataset, batch_size=99, sampler=valid_subsampler)
    
    # Initialize model, optimizer, and loss function
    simple_nn = SimpleNN()
    optimizer = optim.Adam(simple_nn.parameters(), lr=0.01)
    error = nn.BCELoss()
    
    # Training loop for each epoch
    for epoch in range(300):
        
        # Validation Step (No gradient calculation)
        with torch.no_grad():
            valid_loss = 0
            num_right = 0
            for tensor_x, tensor_y in valid_loader:
                tensor_x = tensor_x.float()
                tensor_y = tensor_y.float().reshape(-1, 1)
                output = simple_nn(tensor_x)
                loss = error(output, tensor_y)
                valid_loss += loss.item() * len(tensor_x)
                result = [1 if out >= 0.5 else 0 for out in output]
                num_right += np.sum(np.array(result) == tensor_y.numpy().reshape(-1))
            
            valid_loss = valid_loss / len(valid_loader.sampler.indices)
            valid_accuracy = num_right / len(valid_loader.sampler.indices)
        
            if epoch % 50 == 0:
                print('Valid Loss: {} Accuracy: {} Epoch:{}'.format(valid_loss, valid_accuracy, epoch))

        # Training Step
        train_loss = 0
        num_right = 0
        for tensor_x, tensor_y in train_loader:
            tensor_x = tensor_x.float()
            tensor_y = tensor_y.float().reshape(-1, 1)
            optimizer.zero_grad()
            output = simple_nn(tensor_x)
            loss = error(output, tensor_y)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * len(tensor_x)
            result = [1 if out >= 0.5 else 0 for out in output]
            num_right += np.sum(np.array(result) == tensor_y.numpy().reshape(-1))
        
        train_loss = train_loss / len(train_loader.sampler.indices)
        accuracy = num_right / len(train_loader.sampler.indices)
    
        if epoch % 50 == 0:
            print('Loss: {} Accuracy: {} Epoch:{}'.format(train_loss, accuracy, epoch))

    # Save validation performance for current fold 
    loss_list.append(valid_loss)
    acc_list.append(valid_accuracy)

FOLD 0
--------------------------------
Valid Loss: 1.113208792063111 Accuracy: 0.36312849162011174 Epoch:0
Loss: 2.5942690868056224 Accuracy: 0.5407303370786517 Epoch:0
Valid Loss: 0.5883780181740915 Accuracy: 0.7150837988826816 Epoch:50
Loss: 0.5572928959147984 Accuracy: 0.7275280898876404 Epoch:50
Valid Loss: 0.6362949759600549 Accuracy: 0.6927374301675978 Epoch:100
Loss: 0.5312075767289387 Accuracy: 0.7289325842696629 Epoch:100
Valid Loss: 0.7679927945137024 Accuracy: 0.6871508379888268 Epoch:150
Loss: 0.5290626799373814 Accuracy: 0.7514044943820225 Epoch:150
Valid Loss: 1.0699697626369626 Accuracy: 0.7094972067039106 Epoch:200
Loss: 0.4982989212770141 Accuracy: 0.7640449438202247 Epoch:200
Valid Loss: 0.9463695107891573 Accuracy: 0.7094972067039106 Epoch:250
Loss: 0.4920642075829961 Accuracy: 0.7584269662921348 Epoch:250
FOLD 1
--------------------------------
Valid Loss: 1.1051078686553442 Accuracy: 0.42696629213483145 Epoch:0
Loss: 2.6727289313353797 Accuracy: 0.4964936886395512

In [11]:
# Final summary after all folds are complete
print('Training Ended')
print('Average Loss: {} Average Accuracy: {}'.format(np.mean(loss_list), np.mean(acc_list)))

Training Ended
Average Loss: 1.1216399220938094 Average Accuracy: 0.7093214487477246
